In [1]:
!pip install qwen_vl_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 837.9 kB/s eta 0:00:0000:0100:02


In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

/home/atin/miniconda3/envs/khaitd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.24it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
prompt = """
Identify the pedestrian attributes below:
1. Gender (e.g., male, female)
2. Hair (style, length, and color)
3. Eyeglasses (is the person wearing glasses?) 
4. Upper-body clothing (type and color)
5. Lower-body clothing (type and color)
6. Footwear (type and color)

Based on the extracted information, write a single sentence in English, omit any details that are not visible.
"""

In [18]:
img_path = "/home/atin/ai_t4/khaitd/Image_retrieval/Database/1e94383a-6503-4a5c-b3eb-7423dd1a2226.jpg"

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "/home/atin/ai_t4/khaitd/Image_retrieval/Database/1e94383a-6503-4a5c-b3eb-7423dd1a2226.jpg",
            },
            {"type": "text", "text": prompt},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
inputs = processor(
    text=[text],
    images=Image.open(img_path),
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


['The pedestrian is a female with long black hair, wearing glasses, a black top, black pants, and black shoes.']


In [11]:
#### TEST ####
#### QWEN VL 7B Instruct ####
linkimage_1 = "/home/atin/ai_t4/khaitd/Image_retrieval/Database/0cf57668-fe69-48fc-8da0-251e75289314.jpg" # Đeo kính, áo trắng, quần vàng, giày trắng
linkimage_2 = "/home/atin/ai_t4/khaitd/Image_retrieval/crops/person_5.png" # Áo vàng, tóc vàng
linkimage_3 = "/home/atin/ai_t4/khaitd/Image_retrieval/Database/e9b914f1-b08e-4584-a7b5-65b2373477b0.jpg" # Đeo kính, Áo đen, quần đen
linkimage_4 = "/home/atin/ai_t4/khaitd/Image_retrieval/Database/0fad08ca-e8e1-4e5e-9e80-f68755dc0ee2.jpg" # Áo ngắn màu đen, quần đen, giày trắng
linkimage_5 = "/home/atin/ai_t4/khaitd/Image_retrieval/Database/1fdac95a-f3f8-4eca-aac9-d26ea0217b7a.jpg" # Áo đen, quần đen, dép đen, đeo kính
linkimage_6 = "/home/atin/ai_t4/khaitd/Image_retrieval/Database/2ce5b74f-88dc-4fb8-b9a2-9ffb184e2ab8.jpg" # Phụ nữ, váy đen

text_return_1 = 'The pedestrian is a male with short dark hair, wearing glasses, a white shirt, beige pants, and white sneakers.'
text_return_2 = 'The pedestrian appears to be male with short dark hair, is not wearing glasses, and is dressed in a gray hoodie.'
text_return_3 = 'The pedestrian appears to be male, with short dark hair, is not wearing glasses, is dressed in a dark-colored long-sleeve shirt and pants, and is wearing dark shoes.'
text_return_4 = 'The pedestrian is a male with short dark hair, wearing a dark vest over a light-colored shirt, dark pants, and white sneakers.'
text_return_5 = 'The pedestrian is a male with short black hair, wearing glasses, a black graphic t-shirt, black pants, and black sandals.'
text_return_6 = 'The pedestrian is a female with short dark hair, wearing glasses, dressed in a black top and skirt, and white high-heeled shoes.'

In [ ]:
#### TEST ####
#### QWEN VL 7B Instruct ####
linkimage_1 = "/home/atin/ai_t4/khaitd/Image_retrieval/Database/0cf57668-fe69-48fc-8da0-251e75289314.jpg" # Đeo kính, áo trắng, quần vàng, giày trắng
linkimage_2 = "/home/atin/ai_t4/khaitd/Image_retrieval/crops/person_5.png" # Áo vàng, tóc vàng
linkimage_3 = "/home/atin/ai_t4/khaitd/Image_retrieval/Database/e9b914f1-b08e-4584-a7b5-65b2373477b0.jpg" # Đeo kính, Áo đen, quần đen
linkimage_4 = "/home/atin/ai_t4/khaitd/Image_retrieval/Database/0fad08ca-e8e1-4e5e-9e80-f68755dc0ee2.jpg" # Áo ngắn màu đen, quần đen, giày trắng
linkimage_5 = "/home/atin/ai_t4/khaitd/Image_retrieval/Database/1fdac95a-f3f8-4eca-aac9-d26ea0217b7a.jpg" # Áo đen, quần đen, dép đen, đeo kính
linkimage_6 = "/home/atin/ai_t4/khaitd/Image_retrieval/Database/2ce5b74f-88dc-4fb8-b9a2-9ffb184e2ab8.jpg" # Phụ nữ, váy đen
linkimage_7 = "/home/atin/ai_t4/khaitd/Image_retrieval/Database/1e94383a-6503-4a5c-b3eb-7423dd1a2226.jpg"

text_return_1 = 'The pedestrian is a male with short, dark hair, wearing glasses, a white shirt, khaki pants, and white shoes.'
text_return_2 = 'The pedestrian is a male with short black hair, wearing a grey hoodie and beige pants.'
text_return_3 = 'The pedestrian is a male with short hair, wearing glasses, a dark-colored shirt, dark pants, and sneakers.'
text_return_4 = 'The person is a male with short black hair, wearing a dark-colored shirt and brown pants.'
text_return_5 = 'The pedestrian is a male with short black hair, wearing glasses, a black graphic t-shirt, black pants, and black sandals.'
text_return_6 = 'The pedestrian is a female with short hair, wearing a black dress and high heels.'
text_return_7 = 'The pedestrian is a female with long black hair, wearing glasses, a black top, black pants, and black shoes.'